##### quest
- NSC_BND_M20 연속형 데이터 사용
- 목표변수:SICK_SYM1, 설명변수 5개 사용
- DecisionTreeClassifier 전처리, 학습, 평가
- option : SICK_SYM1 항목은 8개만 선택

### 지도학습

##### 분류 - 다항분류
 - 목적변수의 범주가 2개 이상인것

In [ ]:
import pymongo as mg
import pandas as pd 

In [ ]:
client = mg.MongoClient(host='mongodb://localhost:27017')
database = client['db_NHIS'] #Database 선택
collection = database['Join Collection NSC2_BND_M20'] # collection 선택
cursor = collection.find({})
list_M20 = list(cursor) # casting
df_M20 = pd.DataFrame(list_M20) 
df_M20[:2]

In [ ]:
df_M20.to_csv('../../../datasets/Join Collection NSC2_BND_M20.csv') 

##### 전처리
- 목표변수 : 'SICK_SYM1'
- 설명변수 : 'OPRTN_YN','TOT_PRSC_DD_CNT','ED_RC_TOT_AMT', 'EDC_SBA', 'EDC_INSUR_BRDN_AMT'

In [ ]:
df_M20.columns

In [ ]:
df_M20.info()

In [ ]:
df_M20_extract = df_M20[['SICK_SYM1','OPRTN_YN','TOT_PRSC_DD_CNT','ED_RC_TOT_AMT', 'EDC_SBA', 'EDC_INSUR_BRDN_AMT']]
df_M20_extract.isnull().sum()

In [ ]:

df_M20_extract['SICK_SYM1'].value_counts()

In [ ]:
import re

# 알파벳을 숫자로 변환하는 함수 정의
def alphabet_to_number(s):
    result = ""
    for char in s:
        if re.match(r'[A-Za-z]', char):
            result += str(ord(char) - ord('A') + 1)
        else:
            result += char
    return result

##### 정형화

In [ ]:
target_SICK = df_M20_extract['SICK_SYM1_tonumber'] #목표변수 
features_SICK = df_M20_extract.drop(columns=['SICK_SYM1_tonumber','SICK_SYM1']) #설명변수 
target_SICK.shape, features_SICK.shape

##### 모델학습

In [ ]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier() #인스턴스화 
model.fit(features_SICK, target_SICK) #모델 훈련

In [ ]:
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree

In [ ]:
features_SICK.columns.to_list()

In [ ]:
plot_tree(model, feature_names=features_SICK.columns.to_list(), filled=True) # filled=True
plt.show()

##### 평가

In [ ]:
target_prdict = model.predict(features_SICK)  #설명변수를 통해 목저변수의 값을 예측
len(target_prdict)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(target_SICK, target_prdict) #실제값과 예측한 값을 비교를 통해 잘 예측했는지 평가